# 시작

In [1]:
!git clone https://github.com/uchidalab/time_series_augmentation.git

fatal: destination path 'time_series_augmentation' already exists and is not an empty directory.


In [2]:
# # **************** UNCOMMENT AND RUN THIS CELL IF YOU NEED TO INSTALL/ UPGRADE TSAI ****************
stable = True # Set to True for latest pip version or False for main branch in GitHub
!pip install {"tsai -U" if stable else "git+https://github.com/timeseriesAI/tsai.git"} >> /dev/null

In [3]:
from tsai.all import *
computer_setup()

os             : Linux-5.4.188+-x86_64-with-Ubuntu-18.04-bionic
python         : 3.7.13
tsai           : 0.3.1
fastai         : 2.6.3
fastcore       : 1.4.4
torch          : 1.11.0+cu113
device         : 1 gpu (['Tesla T4'])
cpu cores      : 4
RAM            : 25.46 GB
GPU memory     : [14.75] GB


In [4]:
%cd '/content/time_series_augmentation'

/content/time_series_augmentation


# TS-SMOTE: jitter

In [5]:
import numpy as np
import os
from utils.input_data import read_data_sets
import utils.datasets as ds
import utils.augmentation as aug
import utils.helper as hlp
import matplotlib.pyplot as plt

In [6]:
train = pd.read_csv('/content/drive/MyDrive/AIBIZ/Data/v23/leaktype/leaktype_train.csv')
label_counts = collections.Counter(train['leaktype'])
print('Counts by label:', dict(label_counts))

#
df_out = train[train.leaktype == 'out'] # 10배
df_in = train[train.leaktype == 'in'] # 10배
df_normal = train[train.leaktype == 'normal']
df_other = train[train.leaktype == 'other'] # 2.57
df_noise = train[train.leaktype == 'noise'] # 3.6

X_out = np.array(df_out.drop('leaktype', axis=1))
X_in = np.array(df_in.drop('leaktype', axis=1))
X_other = np.array(df_other.drop('leaktype', axis=1))
X_noise = np.array(df_noise.drop('leaktype', axis=1))



#from sklearn.preprocessing import StandardScaler
#scaler = StandardScaler()

#X_arrays = [X_out, X_in, X_other, X_noise]
#norm_arrays = []
#for el in X_arrays:
#  X_1d = el.reshape(1, -1)
#  scaler = scaler.fit(X_1d.T)
#  normalized_X = scaler.transform(X_1d.T).T
#  norm_X = normalized_X.reshape((el.shape[0], el.shape[1]))
#  norm_arrays.append(norm_X)

X_arrays = [X_out, X_in, X_other, X_noise]
jittered_Xs = {'out':[], 'in':[], 'other':[],'noise':[]}
input_shape = (513, 1)
for x in range(9):

  #for i in range(len(norm_arrays)): # 4번
  for i in range(len(X_arrays)): # 4번
    X_aug = X_arrays[i].reshape((-1, input_shape[0], input_shape[1]))
    X_jitter = aug.jitter(X_aug) 
    X_jitter = X_jitter.reshape((-1,513))
    if i == 0:
      jittered_Xs['out'].append(X_jitter)
    elif i == 1:
      jittered_Xs['in'].append(X_jitter)
    elif i == 2:
      jittered_Xs['other'].append(X_jitter)
    else:
      jittered_Xs['noise'].append(X_jitter)

# jittered_Xs

Counts by label: {'out': 1800, 'in': 1800, 'normal': 18000, 'other': 7000, 'noise': 5000}


In [7]:
jcon_out = np.concatenate(jittered_Xs['out'])
jcon_in = np.concatenate(jittered_Xs['in'])
jcon_other = np.concatenate(jittered_Xs['other'][:1])
jcon_noise = np.concatenate(jittered_Xs['noise'][:2])

In [8]:
jcon_other = np.concatenate([jcon_other, jittered_Xs['other'][2][:4000]])
jcon_other.shape

(11000, 513)

In [9]:
jcon_noise = np.concatenate([jcon_noise, jittered_Xs['noise'][7][:3000]])
jcon_noise.shape

(13000, 513)

In [10]:
df_in_j = pd.DataFrame(jcon_in, columns = train.drop('leaktype', axis=1).columns)
df_in_j['leaktype'] = 'in'

df_out_j = pd.DataFrame(jcon_out, columns = train.drop('leaktype', axis=1).columns)
df_out_j['leaktype'] = 'out'

df_other_j = pd.DataFrame(jcon_other,columns = train.drop('leaktype', axis=1).columns)
df_other_j['leaktype'] = 'other'

df_noise_j = pd.DataFrame(jcon_noise,columns = train.drop('leaktype', axis=1).columns)
df_noise_j['leaktype'] = 'noise'

In [11]:
jitter_final = pd.concat([train, df_in_j, df_out_j, df_other_j, df_noise_j], ignore_index=True)
jitter_final

,leaktype,0HZ,10HZ,20HZ,30HZ,40HZ,50HZ,60HZ,70HZ,80HZ,...,5030HZ,5040HZ,5050HZ,5060HZ,5070HZ,5080HZ,5090HZ,5100HZ,5110HZ,5120HZ
0,out,0.000000,2.000000,2.000000,0.000000,2.000000,0.000000,2.000000,2.000000,2.000000,...,2.000000,5.000000,2.000000,2.000000,5.000000,2.000000,2.000000,5.000000,2.000000,5.000000
1,out,0.000000,0.000000,0.000000,3.000000,0.000000,3.000000,0.000000,0.000000,0.000000,...,0.000000,3.000000,3.000000,3.000000,3.000000,3.000000,3.000000,3.000000,3.000000,6.000000
2,out,0.000000,4.000000,4.000000,4.000000,4.000000,5.000000,4.000000,4.000000,5.000000,...,5.000000,5.000000,6.000000,5.000000,6.000000,6.000000,6.000000,5.000000,6.000000,4.000000
3,out,0.000000,6.000000,5.000000,5.000000,6.000000,5.000000,6.000000,6.000000,5.000000,...,6.000000,6.000000,7.000000,7.000000,5.000000,6.000000,5.000000,5.000000,7.000000,7.000000
4,out,0.000000,3.000000,0.000000,0.000000,3.000000,0.000000,0.000000,3.000000,3.000000,...,3.000000,3.000000,3.000000,3.000000,3.000000,3.000000,3.000000,3.000000,3.000000,3.000000
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
89995,noise,0.040346,-0.021725,-0.024306,-0.007326,-0.053681,4.042830,-0.026590,3.943549,3.967796,...,0.024565,-0.002647,-0.007959,0.017659,0.085779,0.008586,0.005517,0.007568,-0.020010,-0.002026
89996,noise,0.000026,3.018577,-0.011893,-0.007647,0.005525,3.001867,2.987988,2.973897,3.007849,...,9.979038,9.998005,10.005730,10.084565,9.979079,9.985812,10.006386,9.995558,9.957350,6.975459
89997,noise,-0.048243,-0.031808,0.017710,3.019750,2.987716,2.997191,40.025759,3.020094,3.025678,...,2.964418,3.040464,3.008673,3.046393,2.980072,3.020369,3.021783,3.003184,2.933387,3.005946
89998,noise,0.003293,8.002367,7.952224,10.994467,9.043160,9.039337,8.010989,8.996546,7.966730,...,9.997380,9.954834,9.993036,9.006775,8.975099,8.020669,11.999493,10.999163,10.973173,8.014381


In [12]:
# shuffle
from sklearn.utils import shuffle
jitter_final = shuffle(jitter_final)
jitter_final

,leaktype,0HZ,10HZ,20HZ,30HZ,40HZ,50HZ,60HZ,70HZ,80HZ,...,5030HZ,5040HZ,5050HZ,5060HZ,5070HZ,5080HZ,5090HZ,5100HZ,5110HZ,5120HZ
87078,noise,0.005399,10.976302,11.004327,13.025422,11.014046,8.976963,10.972641,10.969316,12.973180,...,13.012112,13.014334,10.965850,13.044342,16.031916,16.010348,10.982522,12.973495,13.058180,12.989470
53197,out,-0.029684,4.018076,4.011465,3.993288,4.960577,4.952697,6.992324,4.035777,4.996101,...,5.042087,4.985781,6.040282,5.036128,5.018804,4.985898,4.957853,5.054740,4.959402,3.963058
84310,noise,-0.036462,1.978458,1.988632,2.026893,2.043919,2.010950,5.005245,2.011974,5.015500,...,2.034276,4.979277,1.973209,2.010046,5.051888,5.004540,5.040738,4.954238,5.040084,4.965861
7170,normal,0.000000,5.000000,5.000000,6.000000,5.000000,5.000000,6.000000,7.000000,6.000000,...,13.000000,13.000000,11.000000,12.000000,13.000000,10.000000,11.000000,9.000000,8.000000,10.000000
17010,normal,0.000000,4.000000,4.000000,4.000000,3.000000,3.000000,5.000000,4.000000,3.000000,...,6.000000,5.000000,5.000000,6.000000,6.000000,6.000000,6.000000,5.000000,6.000000,5.000000
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
3320,in,0.000000,4.000000,4.000000,4.000000,5.000000,6.000000,9.000000,10.000000,12.000000,...,6.000000,5.000000,6.000000,5.000000,5.000000,5.000000,5.000000,6.000000,5.000000,7.000000
73597,other,0.001342,4.042307,2.984358,2.997811,3.044257,2.993894,2.995622,2.974626,3.053818,...,4.997498,4.999966,4.970721,4.049890,5.007747,4.980276,5.011346,4.968800,5.087523,6.014210
53034,out,0.012051,3.987709,4.001404,4.042170,3.981803,4.007163,5.982100,7.964607,9.990427,...,4.042614,4.029501,6.019919,4.006433,4.005867,6.030269,4.046915,6.006544,5.982227,5.975771
35991,in,-0.016973,2.007927,1.972442,1.990647,1.983157,2.031736,1.960356,2.003283,2.040369,...,2.058572,1.986571,5.025132,1.992316,2.004678,1.960346,1.991142,4.986000,4.988650,4.991883
